**Loading packages and cleaning**

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error

In [43]:
aafc_data=pd.read_csv("aafc_data.txt",index_col=None)
aafc_data.head()

,Unnamed: 0,TWP_ID,ECODISTRICT_ID,YEAR,YieldKgAcre,SumPcpn18_20,SumPcpn19_21,SumPcpn20_22,SumPcpn21_23,SumPcpn22_24,...,SoilMoisture29_31,SoilMoisture30_32,SoilMoisture31_33,SoilMoisture32_34,SoilMoisture33_35,SoilMoisture34_36,SoilMoisture35_37,SoilMoisture36_38,SoilMoisture37_39,SoilMoisture38_40
0,0,00101E1,852.0,2010,867.766846,53.6,111.1,109.7,117.9,46.4,...,16.960125,18.766207,17.186998,15.461519,19.738222,22.958089,27.206203,26.480087,28.678156,26.308484
1,1,00101W1,852.0,2010,673.685028,57.2,114.7,110.5,114.0,46.2,...,16.328520,17.926029,16.787544,14.779726,20.245149,23.608204,28.560990,27.324254,29.079177,26.927224
2,2,00101W2,796.0,2010,824.303864,39.0,96.4,109.8,101.2,111.4,...,13.117879,12.869142,12.831834,14.126196,16.385776,18.650751,20.287069,20.514132,19.564788,16.681692
3,3,00102E1,853.0,2010,1006.708496,37.5,158.2,157.8,161.4,46.9,...,17.060778,18.699156,17.345822,15.998957,20.091525,22.761273,26.337430,25.559602,27.611729,25.575794
4,4,00102W1,852.0,2010,869.040283,57.2,114.7,110.5,114.0,46.2,...,16.050993,17.556860,16.612026,14.480150,20.467884,23.893858,29.156274,27.695178,29.255386,27.199097


In [3]:
aafc_data.groupby('ECODISTRICT_ID')['TWP_ID'].nunique()

ECODISTRICT_ID
375.0    24
379.0     1
647.0     1
652.0     3
657.0     5
         ..
851.0    14
852.0    26
853.0     7
854.0    16
855.0     2
Name: TWP_ID, Length: 137, dtype: int64

In [4]:
pd.set_option('display.max_rows', None)
aafc_data.groupby(['ECODISTRICT_ID']).size()

ECODISTRICT_ID
375.0     264
379.0      11
647.0      11
652.0      33
657.0      55
659.0      11
660.0      44
661.0     119
662.0      11
668.0      11
669.0      99
672.0      66
677.0      55
680.0     220
682.0     110
685.0     165
686.0      22
687.0     374
689.0     264
690.0     198
691.0      55
693.0     253
694.0     242
695.0     176
696.0     429
697.0     187
698.0     176
699.0      66
700.0     165
701.0     341
702.0     275
704.0     220
705.0     341
706.0     297
707.0     528
709.0     913
710.0     176
711.0     253
714.0     264
715.0     209
716.0     132
717.0     352
718.0      33
720.0      22
723.0     451
724.0     495
726.0     363
729.0     946
733.0     429
734.0     154
735.0     275
736.0     473
739.0     121
741.0     286
742.0      77
743.0      55
745.0    1100
747.0     209
748.0    1199
749.0     737
751.0     231
752.0    1111
753.0     946
754.0     396
755.0     330
756.0     627
757.0     220
758.0     374
759.0      55
760.0     858
761.0

In [45]:
def ecodistrict_model(ecodistrict):
    data=aafc_data[aafc_data['ECODISTRICT_ID']==ecodistrict]
    records=len(data)
    unique_twnships=data['TWP_ID'].nunique()
    labels = np.array(data['YieldKgAcre'])
    features= data.drop(['YieldKgAcre','YEAR','TWP_ID','Unnamed: 0','ECODISTRICT_ID'], axis = 1)
    feature_list = list(features.columns)
    features = np.array(features)
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.20, random_state = 42)
    scaler = StandardScaler().fit(train_features)
    train_features = scaler.transform(train_features)
    test_features = scaler.transform(test_features)
    
    model = LassoCV(cv=5, random_state=0, max_iter=10000)
    model.fit(train_features, train_labels)
    lasso_best = Lasso(alpha=model.alpha_)
    lasso_best.fit(train_features, train_labels)
    

    r2train=round(lasso_best.score(train_features, train_labels)*100, 2)
    r2test=round(lasso_best.score(test_features, test_labels)*100, 2)
    
    # Training data
    pred_train = lasso_best.predict(train_features)
    mse_train = round(mean_squared_error(train_labels, pred_train,squared=False),2)


    # Test data
    pred = lasso_best.predict(test_features)
    mse_test =round(mean_squared_error(test_labels, pred,squared=False),2)
    
    
    # Calculate the absolute errors
    errors = abs(pred - test_labels)
    # Print out the mean absolute error (mae)
    mae=round(np.mean(errors), 2)
    
    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errors / test_labels)
    # Calculate and display accuracy
    accuracy = round(100 - np.mean(mape),2)
    #coefficients=lasso_best.coef_
    #importance = np.abs(coefficients)
    #for i in range(0,len(importance)):
    #    if importance[i]>0:
    #        print(feature_list[i],importance[i])
    #print(list(zip(lasso_best.coef_, feature_list)))
    #print(list(zip(lasso_best.coef_, feature_list)))

    
    
    return ecodistrict,records,unique_twnships,r2train,r2test,mse_train,mse_test,mae,accuracy

    

In [ ]:
import warnings
warnings.filterwarnings('ignore')
print(ecodistrict_model(735))

In [41]:
print(ecodistrict_model(375))

SumPcpn24_26 51.17899330798409
SumPcpn25_27 1.7028483301000161
SumPcpn30_32 18.500416413388695
SumPcpn36_38 1.7483154265687901
SumEGDD_C26_28 34.787200110483276
SumEGDD_C28_30 27.28879735892798
SumEGDD_C30_32 40.92016747318157
SumEGDD_C36_38 45.56929662675967
SumHeatD19_21 1.514745636986151
SumHeatD22_24 81.94715988116981
SumHeatD28_30 6.867410954139527
SumHeatD37_39 8.691957130812243
SumFrostD35_37 35.301181797159266
AvgSI18_20 31.343396919821085
AvgSI29_31 25.519123979363105
AvgPrcnAWHC19_21 7.801779286106363
AvgPrcnAWHC20_22 49.222681595438374
NDVI18_20 37.098999680868104
NDVI25_27 17.193573727945044
NDVI27_29 5.02038930970766
NDVI29_31 40.15017812546701
SoilMoisture30_32 3.9293586602658523
SoilMoisture37_39 1.7897636886112573
SoilMoisture38_40 15.627544633599568
(375, 264, 24, 88.25, 85.09, 105.35, 126.22, 97.29, 84.64)


In [40]:
import warnings
warnings.filterwarnings('ignore')
print(ecodistrict_model(735))

SumPcpn18_20 9.503791769358482
SumPcpn21_23 8.957528783562802
SumPcpn22_24 52.215440561200275
SumPcpn33_35 33.13417460879933
SumPcpn38_40 31.184620455410315
SumEGDD_C21_23 3.46973797143348
SumEGDD_C28_30 46.729854988549874
SumEGDD_C29_31 10.393097856594977
SumEGDD_C38_40 13.888135566725053
SumHeatD19_21 6.539206278775213
SumHeatD25_27 4.9844127672507685
SumHeatD31_33 30.229139542448973
NDVI20_22 6.812723863510257
NDVI25_27 4.178045596240265
NDVI26_28 26.099593213404773
NDVI30_32 17.623712578638862
NDVI32_34 24.229125625904675
SoilMoisture24_26 53.31104718420655
SoilMoisture29_31 2.2554542760476046
SoilMoisture32_34 19.892774477191505
SoilMoisture33_35 40.263108770958404
SoilMoisture35_37 5.389916452857087
(735, 275, 25, 76.3, 64.05, 80.22, 81.43, 62.43, 92.91)


In [46]:
ecodistrict_id_lst=[]
r2train_lst=[]
r2test_lst=[]
mse_train_lst=[]
mse_test_lst=[]
mae_lst=[]
accuracy_lst=[]
records_lst=[]
unique_twnships_lst=[]


for i in aafc_data['ECODISTRICT_ID'].unique():
    ecodistrict_id,records,unique_twnships,r2train,r2test,mse_train,mse_test,mae,accuracy=ecodistrict_model(i)
    ecodistrict_id_lst.append(ecodistrict_id)
    r2train_lst.append(r2train)
    r2test_lst.append(r2test)
    mse_train_lst.append(mse_train)
    mse_test_lst.append(mse_test)
    mae_lst.append(mae)
    accuracy_lst.append(accuracy)
    records_lst.append(records)
    unique_twnships_lst.append(unique_twnships)

model_eval=pd.DataFrame()
model_eval['ecodistrict_id']= ecodistrict_id_lst
model_eval['records']=records_lst
model_eval['unique_twnships']=unique_twnships_lst

model_eval['r2train']=r2train_lst
model_eval['r2test']=r2test_lst
model_eval['mse_train']=mse_train_lst
model_eval['mse_test']=mse_test_lst
model_eval['mae']=mae_lst
model_eval['accuracy']=accuracy_lst


In [47]:
model_eval.to_csv("model_eval_lasso.csv")

In [ ]:
model_eval